<a href="https://colab.research.google.com/github/AndresPautrat/MachineLearning_UPC/blob/main/Plotly2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install plotly==4.6.0

     |████████████████████████████████| 7.2MB 2.7MB/s 
  Found existing installation: plotly 3.10.0
    Uninstalling plotly-3.10.0:
      Successfully uninstalled plotly-3.10.0


In [ ]:
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA

from sklearn.naive_bayes import CategoricalNB

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
fileUp = files.upload()
data_set = pd.read_csv(io.BytesIO(fileUp['confirmed_global.csv']))

Saving confirmed_global.csv to confirmed_global (1).csv


In [ ]:
dt=data_set.copy()

In [ ]:
dt.sort_values(by=['4/20/20'], ascending=False)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20
225,NaN,US,37.090200,-95.712900,1,1,2,2,5,5,5,5,5,7,8,8,11,11,11,11,11,11,11,11,12,12,13,13,13,13,13,13,13,13,15,15,15,51,51,57,...,4632,6421,7783,13747,19273,25600,33276,43843,53736,65778,83836,101657,121465,140909,161831,188172,213242,243622,275367,308650,336802,366317,397121,428654,462780,496535,526396,555313,580619,607670,636350,667592,699706,732197,758809,784326,811865,840351,869170,905358
201,NaN,Spain,40.000000,-4.000000,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,6,13,...,9942,11748,13910,17963,20410,25374,28768,35136,39885,49515,57786,65719,73235,80110,87956,95923,104118,112065,119199,126168,131646,136675,141942,148220,153222,158273,163027,166831,170099,172541,177644,184948,190839,191726,198674,200210,204178,208389,213024,219764
137,NaN,Italy,43.000000,12.000000,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,20,62,155,229,322,453,...,27980,31506,35713,41035,47021,53578,59138,63927,69176,74386,80589,86498,92472,97689,101739,105792,110574,115242,119827,124632,128948,132547,135586,139422,143626,147577,152271,156363,159516,162488,165155,168941,172434,175925,178972,181228,183957,187327,189973,192994
116,NaN,France,46.227600,2.213700,0,0,2,3,3,3,4,5,5,5,6,6,6,6,6,6,6,11,11,11,11,11,11,11,12,12,12,12,12,12,12,12,12,12,14,18,...,6633,7652,9043,10871,12612,14282,16018,19856,22304,25233,29155,32964,37575,40174,44550,52128,56989,59105,64338,68605,70478,74390,78167,82048,86334,90676,93790,120633,124298,129257,132473,144944,146923,146906,151808,154188,156921,154715,157026,158636
120,NaN,Germany,51.000000,9.000000,0,0,0,0,0,1,4,4,4,5,8,10,12,12,12,12,13,13,14,14,16,16,16,16,16,16,16,16,16,16,16,16,16,16,17,27,...,7272,9257,12327,15320,19848,22213,24873,29056,32986,37323,43938,50871,57695,62095,66885,71808,77872,84794,91159,96092,100123,103374,107663,113296,118181,122171,124908,127854,130072,131359,134753,137698,141397,143342,145184,147065,148291,150648,153129,154999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
259,Saint Pierre and Miquelon,France,46.885200,-56.315900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
78,Tibet,China,31.692700,88.092400,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
238,Recovered,Canada,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
dt=dt.drop(["Province/State","Country/Region","Lat","Long"],axis=1)

In [ ]:
negatives=[]
for i in range(dt.shape[0]):
  for j in range(dt.shape[1]):
    if(dt.iloc[i,j]<0):
      print(i)
      negatives+=[i]
      break;

37
41
231


In [ ]:
dt = dt.drop(negatives)
dt = dt.drop([38])

In [ ]:
dt_temp=pd.DataFrame()
for i in range(0,dt.shape[1],7):
  dt_temp["Semana " +str(i/7)]=dt.iloc[:,i]
dt=dt_temp.copy()

In [ ]:
dt_semanal=dt.copy()
dt.shape
for i in range(dt.shape[0]):
  for j in range(1,dt.shape[1]):
    dt_semanal.iloc[i,j]=dt.iloc[i,j]-dt.iloc[i,j-1]
dt=dt_semanal.copy()

In [ ]:
dt.iloc[:,:]=normalize(dt.iloc[:,:])

In [ ]:
temp=[]

for i in range(dt.shape[0]):
  if(dt.iloc[i,13]-dt.iloc[i,12]>0):
    temp+=["Aumenta"]
  elif(dt.iloc[i,13]-dt.iloc[i,12]<0):
    temp+=["Disminuye"]
  else:
    temp+=["Igual"]
dt_temp["Target"]=temp
dt=dt_temp

In [ ]:
dt=dt.drop(["Semana 13.0"],axis=1)

In [ ]:
x = dt.iloc[:,:-1].values
y = dt.iloc[:,-1].values

In [ ]:
traces = []
total = sum(dt["Target"].value_counts())
percentage = [(dt["Target"].value_counts()[i]/total)*100 for i in range(3)]
langs = dt["Target"].value_counts().keys().tolist()
traces += [Pie(labels= langs, values= percentage,name='Target')]
dataG = Data(traces)
layout = Layout(barmode="overlay",title="distribución del control contra el covid-19 entre semanas")
fig = Figure(data = dataG,layout= layout)
fig.show()

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:40: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




In [ ]:
dt_semanal=dt_semanal.sort_values(by=['Semana 13.0'], ascending=False)

In [ ]:
dt_temp_trans=dt_semanal.transpose() 

In [ ]:
#dt_temp=pd.DataFrame(dt_semanal.iloc[-4:-1,:])
#temp=dt_semanal.shape[0]//2
#dt_temp=dt_temp.append(dt_semanal.iloc[temp-3:temp,:])
#dt_temp=dt_temp.append(dt_semanal.iloc[:3,:])

In [ ]:
#dt_temp_trans=dt_temp.transpose() 

In [ ]:
fig = go.Figure()
count=0
for i in dt_temp_trans:
  fig.add_trace(go.Scatter(x=dt_temp_trans.index.values, y=dt_temp_trans[i],
                    mode='lines+markers',
                    name=data_set["Country/Region"][i]+"-"+str(i)))
  count+=1

fig.show()

In [ ]:
temp_dt=data_set.copy()
temp_dt=temp_dt.drop(negatives)

In [ ]:
fig = px.scatter_geo(temp_dt, lat="Lat", lon="Long",
                     size="4/24/20",
                     )
fig.show()

In [ ]:
total_semana=[]
for i in dt_semanal:
  total_semana+=[sum(dt_semanal[i])]
total_semana

[555,
 5611,
 21469,
 17586,
 30418,
 5749,
 13732,
 30755,
 88910,
 252752,
 464625,
 547102,
 574986,
 567772]

In [ ]:
dt_semanal.loc[2]

Semana 0.0       0
Semana 1.0       0
Semana 2.0       0
Semana 3.0       0
Semana 4.0       0
Semana 5.0       1
Semana 6.0      11
Semana 7.0       8
Semana 8.0      54
Semana 9.0     228
Semana 10.0    545
Semana 11.0    725
Semana 12.0    588
Semana 13.0    750
Name: 2, dtype: int64

In [ ]:
temp_graphs=[]
for i in dt_semanal.index:
  temp_graphs+=[go.Bar(name=data_set["Country/Region"][i]+"-"+str(i), x=dt_semanal.columns, y=dt_semanal.loc[i,:])]
fig = go.Figure(data=temp_graphs)
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='SF Zoo', x=dt_semanal.columns, y=dt_semanal.iloc[1,:]),
    go.Bar(name='LA Zoo', x=dt_semanal.columns, y=dt_semanal.iloc[1,:])
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()